In [13]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
import nltk

In [14]:
df = pd.read_table('yelp_labelled.txt',header=None,names=['review','sentiment'])

In [15]:
df.head()

,review,sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [29]:
from nltk import word_tokenize
tokens = word_tokenize(df.head()['review'][0])
sentences = df.apply(lambda row: word_tokenize(row['review']), axis=1)
dataset = [(words, label) for words, label in zip(sentences, df['sentiment'])]

In [30]:
from nltk.sentiment import SentimentAnalyzer
sentim_analyzer = SentimentAnalyzer()


In [60]:
from nltk.sentiment.util import *
all_words_neg = sentim_analyzer.all_words([mark_negation(doc) for doc in dataset])
unigram_feats = sentim_analyzer.unigram_word_feats(all_words_neg, min_freq=4)

In [51]:
sentim_analyzer.add_feat_extractor(extract_unigram_feats, unigrams=unigram_feats)
training_set = sentim_analyzer.apply_features(dataset[:800])
test_set = sentim_analyzer.apply_features(dataset[801:])

In [53]:
from nltk.classify import NaiveBayesClassifier
trainer = NaiveBayesClassifier.train
classifier = sentim_analyzer.train(trainer, training_set)

Training classifier


In [54]:
for key,value in sorted(sentim_analyzer.evaluate(test_set).items()):
    print('{0}: {1}'.format(key, value))

Evaluating NaiveBayesClassifier results...
Accuracy: 0.758793969849
F-measure [0]: 0.818181818182
F-measure [1]: 0.641791044776
Precision [0]: 0.955752212389
Precision [1]: 0.5
Recall [0]: 0.715231788079
Recall [1]: 0.895833333333
